In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import tensorflow as tf

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Activation
import seaborn as sns

# HYPER-PARAMETERS

In [12]:
TEST_SIZE = 0.2

# Optimization

In [13]:
def reduce_mem_usage(df : pd.DataFrame):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo('i1').min and c_max < np.iinfo('i1').max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo('i2').min and c_max < np.iinfo('i2').max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo('i4').min and c_max < np.iinfo('i4').max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo('i8').min and c_max < np.iinfo('i8').max:
                df[col] = df[col].astype(np.int64)
        elif str(col_type)[:8] == 'datetime':
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Потребление памяти меньше на',
         round(start_mem - end_mem, 2),
         'Мб (минус',
         round(100 * (start_mem - end_mem) / start_mem, 1),
         '%)')
    return df

# Data extract

In [11]:
methods = {
    "SHG Intensity" : [
        "SHG Intensity Mean",
        "SHG Intensity MAD",
        "SHG Intensity Contrast",
        "SHG Intensity Correlation",
        "SHG Intensity Entropy",
        "SHG Intensity ASM",
        "SHG Intensity IDM"
    ],
    "R-Ratio" : [
        "R-Ratio Mean",
        "R-Ratio MAD",
        "R-Ratio Contrast",
        "R-Ratio Correlation",
        "R-Ratio Entropy",
        "R-Ratio ASM"
    ],
    "Degree of Circular Polarization" : [
        "Degree of Circular Polarization Mean",
        "Degree of Circular Polarization MAD",
        "Degree of Circular Polarization Contrast",
        "Degree of Circular Polarization Correlation",
        "Degree of Circular Polarization Entropy",
        "Degree of Circular Polarization ASM",
        "Degree of Circular Polarization IDM"
    ],
    "SHG-CD" : [ 
        "SHG-CD MAD",
        "SHG-CD Contrast",
        "SHG-CD Correlation",
        "SHG-CD Entropy",
        "SHG-CD ASM",
        "SHG-CD IDM"
    ],
    "SHG-LD" : [
        "SHG-LD MAD",
        "SHG-LD Contrast",
        "SHG-LD Correlation",
        "SHG-LD Entropy",
        "SHG-LD ASM",
        "SHG-LD IDM"
    ]
}

x_axis = sum([methods[key] for key in methods.keys()], [])
# x_axis += "Pixel Density"

y_axis = "2-Group Tag"

def getData(table_number):
    data = pd.read_excel(io="../../Data/41598_2022_13623_MOESM3_ESM.xlsx", 
    sheet_name=f"{1 << 2 * (table_number - 1)} Subimage Training")
    data = reduce_mem_usage(data)
    data = (data - data.min()) / (data.max() - data.min())
    #data["2-Group Tag"] = data[y_axis] == 2
    return data


# Init data

In [11]:
data = getData(4)

train_data = data.sample(frac=0.8, random_state=0)
test_data = data.drop(train_data.index)
X_train, X_test = train_data[x_axis].to_numpy(), test_data[x_axis].to_numpy()
y_train, y_test = train_data[y_axis].to_numpy(), test_data[y_axis].to_numpy()

X_train_data_true = X_train[y_train == 1]
X_train_data_false = X_train[y_train == 0]

n_samples, n_features, batch_size = *X_train.shape, 128

Потребление памяти меньше на 0.24 Мб (минус 75.3 %)


In [12]:
model = Sequential()


model.add(tf.keras.Input(shape=(32,)))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

print(X_train.shape, y_train.shape)
model.fit(X_train, y_train, batch_size=batch_size, epochs=100, validation_data=(X_test, y_test))

y_predict = model.predict(X_test)

(963, 32) (963,)
Epoch 1/100
8/8 [==============================] - 1s 32ms/step - loss: 0.7402 - accuracy: 0.5171 - val_loss: 0.7404 - val_accuracy: 0.5062
Epoch 2/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7351 - accuracy: 0.5213 - val_loss: 0.7345 - val_accuracy: 0.4979
Epoch 3/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7304 - accuracy: 0.5234 - val_loss: 0.7296 - val_accuracy: 0.4979
Epoch 4/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7263 - accuracy: 0.5306 - val_loss: 0.7251 - val_accuracy: 0.5021
Epoch 5/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7226 - accuracy: 0.5348 - val_loss: 0.7209 - val_accuracy: 0.5104
Epoch 6/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7188 - accuracy: 0.5400 - val_loss: 0.7171 - val_accuracy: 0.5062
Epoch 7/100
8/8 [==============================] - 0s 5ms/step - loss: 0.7154 - accuracy: 0.5410 - val_loss: 0.7138 - val_accuracy: 0.5104
Epoch 8/1

In [13]:
tf.math.confusion_matrix(y_test, np.round(y_predict))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[107,   9],
       [ 39,  86]])>

In [14]:
np.round(y_predict)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],

In [15]:
# data[y_axis] = data[y_axis].astype('object')
# sns.pairplot(data, hue=y_axis, vars=methods["SHG Intensity"])